# Building a Spam Filter
** by Gerard Tieng **

For this project, we will be using machine learning and the multinomial Naive Bayes algorithm to classify strings in dataset of more than [5,000 SMS messages](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection) as spam or not spam (ham).

## Data Load and Inspection

In [1]:
import pandas as pd

sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

In [2]:
sms.shape

(5572, 2)

In [3]:
sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
sms['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

From our initial inspection of this dataset, we see that we have 5572 SMS records, 13% of which is spam.

## Train/Test Split

Before we begin the machine learning portion of this project, we will perform a split of the data into a training set and a testing set. The training set will be used to formulate the final algorithm during machine learning, while the testing set remains independent in order to confirm the accuracy of the algorithm,

We'll used `Dataframe.sample` with the argument `frac=1` to randomize the entire dataframe. Meanwhile, we will split the dataset 80/20 to maintain the relative spam/ham ratio of the original dataset (though it is a common splitting ratio when performing machine learning).

In [5]:
random_dataset = sms.sample(frac=1, random_state=1)
split_index = round(random_dataset.shape[0] * 0.8)

In [6]:
train_dataset = random_dataset[:split_index].reset_index()
test_dataset = random_dataset[split_index:].reset_index()

In [7]:
train_dataset.shape, test_dataset.shape

((4458, 3), (1114, 3))

An 80/20 split of the original dataset gives us 4458 records to train our algorithm, and 1114 to confirm our results during testing.

## String Cleaning

In order to properly utilize the multinominal Naive Bayes algorithm effectively, each word within out dataset must be accounted for appropriately. Light string cleaning for word uniformity will be performed here in the process of eliminating punctuation and lowercasing each of the words.

In [8]:
train_dataset["SMS"] = train_dataset['SMS'].str.replace('\W', ' ').str.lower()
test_dataset["SMS"] = test_dataset['SMS'].str.replace('\W', ' ').str.lower()

In [9]:
train_dataset.head()

,index,Label,SMS
0,1078,ham,yep by the pretty sculpture
1,4028,ham,yes princess are you going to make me moan
2,958,ham,welp apparently he retired
3,4642,ham,havent
4,4674,ham,i forgot 2 ask ü all smth there s a card on ...


## Tokenization

Now that our message strings contain uniform words to count, we will transform the entire dataset into a dataframe in which every word within each string will be tallied against every possible word that appears in the entire dataset.

The following code will iterate through each string and identify unique words to construct the master vocabulary.

In [10]:
train_dataset["SMS"] = train_dataset["SMS"].str.split()

In [11]:
vocabulary = []
for row in train_dataset["SMS"]:
    for word in row:
        if word not in vocabulary:
            vocabulary.append(word)
            
vocabulary[0:5]

['yep', 'by', 'the', 'pretty', 'sculpture']

The following code builds the dictionary structure to tally each word once we iterate through all the strings. We will concatinate the two sets of data into one dataframe to build our final tokenized training set.

In [12]:
word_counts_per_sms = {unique_word: [0] * len(train_dataset['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(train_dataset['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [13]:
tokenized = pd.DataFrame(word_counts_per_sms)

In [14]:
training_input = pd.concat([train_dataset, tokenized], axis=1)
training_input.head()

,index,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,1078,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4028,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,958,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4642,ham,[havent],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4674,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Essential Calculations

The multinominal Naive Bayes algorithm will require a few predetermined inputs in order to process:
- **P(Spam)**, the probability of spam
- **P(Ham)**, the probability of ham
- **N(Vocabulary)**, the number of words in the master vocabulary
- **N(Ham,)**, the number of words (including duplicates) in the ham dataset
- **N(Spam)**, the number of words (including duplicates) in the spam dataset
- **Alpha**, the smoothing coefficient to offset any zero values during multiplication.


All these will be plugged into the equation as follows:

![](https://render.githubusercontent.com/render/math?math=P%28w_i%7CSpam%29%20%3D%20%5Cfrac%7BN_%7Bw_i%7CSpam%7D%20%2B%20%5Calpha%7D%7BN_%7BSpam%7D%20%2B%20%5Calpha%20%5Ccdot%20N_%7BVocabulary%7D%7D&mode=display)

P(wi|Spam) will be calculated in a step shown later.

In [16]:
ham_messages = training_input[training_input["Label"] == "ham"]
spam_messages = training_input[training_input["Label"] == "spam"]

In [17]:
p_ham = len(ham_messages) / len(train_dataset)
p_spam = len(spam_messages) / len(train_dataset)

print(p_ham)
print(p_spam)

0.8654104979811574
0.13458950201884254


In [18]:
n_vocabulary = 0
n_ham = 0
n_spam = 0

for sms in train_dataset["SMS"]:
    n_vocabulary += len(sms)

for sms in ham_messages["SMS"]:
    n_ham += len(sms)
    
for sms in spam_messages["SMS"]:
    n_spam += len(sms)    
    
print(n_vocabulary)
print(n_ham)
print(n_spam)

72427
57237
15190


In [19]:
alpha = 1

With all of our predetermined calculations in place, we'll finally used this step to transform each value our tokenized traning set to their weighted values.

In [20]:
param_ham = {}
param_spam = {}

for unique_word in vocabulary:
    param_ham[unique_word] = 0
    param_spam[unique_word] = 0

In [21]:
for unique_word in vocabulary:
    n_word_given_ham = ham_messages[unique_word].sum()
    parameter_ham = (n_word_given_ham + alpha) / ((n_ham + alpha) * n_vocabulary)
    param_ham[unique_word] += parameter_ham
    
    n_word_given_spam = spam_messages[unique_word].sum()
    parameter_spam = (n_word_given_spam + alpha) / ((n_spam + alpha) * n_vocabulary)
    param_spam[unique_word] += parameter_spam

## Testing the Algorithm

Classification is ready to go. The following function is designed to take a raw string, then clean, scan, and multiply each word according to their weights from the parameter dataframe.

In [28]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in param_spam:
            p_spam_given_message *= param_spam[word]
        if word in param_ham:
            p_ham_given_message *= param_ham[word]
        

    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        return "ham"
    elif p_ham_given_message < p_spam_given_message:
        return "spam"
    else:
        return "too close to call"

In [29]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.0169723085724164e-67
P(Ham|message): 1.1123566091769269e-70


'spam'

In [24]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 4.217099313728267e-58
P(Ham|message): 8.609337650711888e-55
Label: Ham


Now, we'll apply the function to each of the strings in our independent testing dataset to confirm the accuracy of the spam filter.

In [30]:
test_dataset["predicted"] = test_dataset["SMS"].apply(classify)

P(Spam|message): 6.026744678575397e-59
P(Ham|message): 9.930121007917125e-53
P(Spam|message): 2.3489817524048205e-76
P(Ham|message): 5.553390107221829e-72
P(Spam|message): 1.412726946645114e-218
P(Ham|message): 2.0217838926063728e-237
P(Spam|message): 2.7222586998890842e-76
P(Ham|message): 8.508608977162041e-72
P(Spam|message): 6.858310526721547e-162
P(Ham|message): 5.34000524732486e-154
P(Spam|message): 5.117559773550259e-260
P(Ham|message): 2.5115360643042542e-242
P(Spam|message): 2.8907518578336707e-17
P(Ham|message): 3.780485756190092e-15
P(Spam|message): 5.0147689685115816e-110
P(Ham|message): 5.354022387518304e-106
P(Spam|message): 1.855159590579861e-103
P(Ham|message): 1.981213810796845e-98
P(Spam|message): 4.087162570744061e-39
P(Ham|message): 6.129374039877656e-39
P(Spam|message): 1.2729958122074796e-34
P(Ham|message): 1.1117355469924229e-31
P(Spam|message): 3.010326908897988e-107
P(Ham|message): 2.1687825851251843e-101
P(Spam|message): 1.2892870345224454e-234
P(Ham|message): 

In [45]:
test_dataset.head()

,index,Label,SMS,predicted
0,2131,ham,later i guess i needa do mcat study too,ham
1,3418,ham,but i haf enuff space got like 4 mb,ham
2,3424,spam,had your mobile 10 mths update to latest oran...,spam
3,1538,ham,all sounds good fingers makes it difficult ...,ham
4,5393,ham,all done all handed in don t know if mega sh...,ham


In [46]:
correct = len(test_dataset[test_dataset["Label"] == test_dataset["predicted"]])

In [48]:
accuracy = correct / len(test_dataset)
accuracy

0.9640933572710951

Using our spam filter based on the multinomial Naive Bayes algorithm, we were able to correctly predict spam from ham in the testing dataset of more than 1000 SMS records with 96.4% accuracy!